<a href="https://colab.research.google.com/github/FRED984/spam_sms_classifier/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

In [ ]:
train_data = pd.read_csv(train_file_path, sep="\t", header=0, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep="\t", header=0, names=["label", "message"])
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['message'])
y_train = LabelEncoder().fit_transform(train_data['label'].values)
X_test = vectorizer.transform(test_data['message'])
y_test= LabelEncoder().fit_transform(test_data['label'].values)

In [ ]:

# Define the neural network model
#model = keras.models.Sequential([
    #keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    #keras.layers.Dense(32, activation='relu'),
    #keras.layers.Dense(1, activation='softmax')])

# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
#model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#print("Test accuracy:", accuracy)


In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the parameter grid to search
param_grid = {'alpha': [0.05,0.1, 0.5, 1.0, 2.0]}

# Initialize the MultinomialNB model
nb_model = MultinomialNB()

# Perform Grid Search Cross Validation
grid_search = GridSearchCV(estimator=nb_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Retrain the model with the best parameters
model = MultinomialNB(alpha=best_params['alpha'])
model.fit(X_train, y_train)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  vectorized_message = vectorizer.transform([pred_text])
  prediction = model.predict_proba(vectorized_message)
  #prediction = model.predict(vectorized_message)
  likeliness = prediction [0][1]
  result = "spam" if likeliness > 0.5 else "ham"
  prediction = [likeliness, result]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
